In [3]:
# imports
import os
import pickle
import numpy as np
import pandas as pd
import glob
import json
from rich import print as rprint
from rich.table import Table
from rich.console import Console
import gzip
import datasets
from datasets import load_dataset

datasets.logging.set_verbosity_error()

### Global Functions

In [4]:
def display_pickle_summary(data, title="Pickle File Contents"):
    """
    Load and summarize the contents of a pickle file using rich.

    :param data: .pkl data
    :param title: Optional title for the printed table.
    """
    table = Table(title=title)
    
    table.add_column("Key/Type", style="cyan", no_wrap=True)
    table.add_column("Description", style="magenta")

    total_size = 0
    if isinstance(data, dict):
        for key, value in data.items():
            desc = f"{type(value).__name__}, len={len(value)}" if hasattr(value, '__len__') else type(value).__name__
            table.add_row(str(key), desc)
            if key in ["train", "test", "val"]:
                total_size += len(value)
            else:
                total_size = "N/A"
    else:
        table.add_row(type(data).__name__, f"{data}" if isinstance(data, (int, float, str)) else str(type(data)))

    table.add_row("Total Size", str(total_size))
    console = Console()
    console.print(table)
    
    if "train" in data or "test" in data or "val" in data:
        rprint("Train Sample:")
        rprint(data['train'][0])
        rprint("Val Sample:")
        rprint(data['val'][0])
        rprint("Test Sample:")
        print(data['test'][0])
        
        
def df_stats(df: pd.DataFrame, title="DataFrame Stats"):
    table = Table(title=title)
    rprint(f"DataFrame shape: {df.shape}")
    table.add_column("Column", style="cyan", no_wrap=True)
    table.add_column("Non-Null Count", style="yellow")
    table.add_column("Unique Count", style="magenta")
    table.add_column("Null/NA Count", style="red")
    table.add_column("Data Type", style="green")

    for col in df.columns:
        try:
            non_null_count = df[col].notna().sum()
        except:
            non_null_count = "Error"
        try:
            unique_count = df[col].nunique(dropna=True)
        except:
            unique_count = "Error"
        try:
            null_count = df[col].isna().sum()
        except:
            null_count = "Error"
        try:
            dtype = str(df[col].dtype)
        except:
            dtype = "Error"
        table.add_row(col, str(non_null_count), str(unique_count), str(null_count), dtype)

    Console().print(table)

### Loading Data from HuggingFace

#### Review Samples

In [4]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", 
                       "raw_review_All_Beauty", 
                       trust_remote_code=True)

In [6]:
dataset["full"][0]

{'rating': 5.0,
 'title': 'Such a lovely scent but not overpowering.',
 'text': "This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!",
 'images': [],
 'asin': 'B00YQ6X8EO',
 'parent_asin': 'B00YQ6X8EO',
 'user_id': 'AGKHLEW2SOWHNMFQIJGBECAF7INQ',
 'timestamp': 1588687728923,
 'helpful_vote': 0,
 'verified_purchase': True}

#### Metadata

In [8]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", 
                       "raw_meta_All_Beauty", 
                       split="full", 
                       trust_remote_code=True)

meta_All_Beauty.jsonl:   0%|          | 0.00/213M [00:00<?, ?B/s]

Generating full split:   0%|          | 0/112590 [00:00<?, ? examples/s]

In [9]:
dataset[0]

{'main_category': 'All Beauty',
 'title': 'Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)',
 'average_rating': 4.8,
 'rating_number': 10,
 'features': [],
 'description': [],
 'price': 'None',
 'images': {'hi_res': [None,
   'https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg'],
  'large': ['https://m.media-amazon.com/images/I/41qfjSfqNyL.jpg',
   'https://m.media-amazon.com/images/I/41w2yznfuZL.jpg'],
  'thumb': ['https://m.media-amazon.com/images/I/41qfjSfqNyL._SS40_.jpg',
   'https://m.media-amazon.com/images/I/41w2yznfuZL._SS40_.jpg'],
  'variant': ['MAIN', 'PT01']},
 'videos': {'title': [], 'url': [], 'user_id': []},
 'store': 'Howard Products',
 'categories': [],
 'details': '{"Package Dimensions": "7.1 x 5.5 x 3 inches; 2.38 Pounds", "UPC": "617390882781"}',
 'parent_asin': 'B01CUPMQZE',
 'bought_together': None,
 'subtitle': None,
 'author': None}

#### Pure IDs Files (Before Splitting)

In [10]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", 
                       "5core_rating_only_All_Beauty", 
                       trust_remote_code=True)

All_Beauty.csv:   0%|          | 0.00/147k [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

In [11]:
dataset['full'][0:5]

{'user_id': ['AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA'],
 'parent_asin': ['B07J3GH1W1',
  'B07W397QG4',
  'B07KG1TWP5',
  'B08JTNQFZY',
  'B07SLFWZKN'],
 'rating': ['5.0', '5.0', '5.0', '5.0', '3.0'],
 'timestamp': ['1547589356557',
  '1593352422858',
  '1596473351088',
  '1617904219785',
  '1619737501209']}

#### Pure IDs Files (After Splitting)

In [12]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", 
                       "5core_timestamp_All_Beauty", 
                       trust_remote_code=True)

All_Beauty.train.csv:   0%|          | 0.00/130k [00:00<?, ?B/s]

All_Beauty.valid.csv:   0%|          | 0.00/16.1k [00:00<?, ?B/s]

All_Beauty.test.csv:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [13]:
dataset['train'][:5]

{'user_id': ['AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA'],
 'parent_asin': ['B07J3GH1W1',
  'B07W397QG4',
  'B07KG1TWP5',
  'B08JTNQFZY',
  'B07SLFWZKN'],
 'rating': ['5.0', '5.0', '5.0', '5.0', '3.0'],
 'timestamp': ['1547589356557',
  '1593352422858',
  '1596473351088',
  '1617904219785',
  '1619737501209']}

#### Need additional user historical interactions

In [14]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", 
                       "5core_timestamp_w_his_All_Beauty", 
                       trust_remote_code=True)

All_Beauty.train.csv:   0%|          | 0.00/291k [00:00<?, ?B/s]

All_Beauty.valid.csv:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

All_Beauty.test.csv:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [17]:
dataset['train'][:5]

{'user_id': ['AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA',
  'AFSKPY37N3C43SOI5IEXEK5JSIYA'],
 'parent_asin': ['B07J3GH1W1',
  'B07W397QG4',
  'B07KG1TWP5',
  'B08JTNQFZY',
  'B07SLFWZKN'],
 'rating': ['5.0', '5.0', '5.0', '5.0', '3.0'],
 'timestamp': ['1547589356557',
  '1593352422858',
  '1596473351088',
  '1617904219785',
  '1619737501209'],
 'history': ['',
  'B07J3GH1W1',
  'B07J3GH1W1 B07W397QG4',
  'B07J3GH1W1 B07W397QG4 B07KG1TWP5',
  'B07J3GH1W1 B07W397QG4 B07KG1TWP5 B08JTNQFZY']}

### Saving 5-Core

In [3]:
from tqdm import tqdm
from pprint import pprint
from huggingface_hub import hf_hub_download

In [23]:
rating_file = hf_hub_download(
            repo_id='McAuley-Lab/Amazon-Reviews-2023',
            filename='benchmark/5core/rating_only/Toys_and_Games.csv',
            repo_type='dataset'
            )

data_file = hf_hub_download(
            repo_id='McAuley-Lab/Amazon-Reviews-2023',
            filename='raw/review_categories/Toys_and_Games.jsonl',
            repo_type='dataset'
            )

Toys_and_Games.csv:   0%|          | 0.00/224M [00:00<?, ?B/s]

'/home/scur2745/.cache/huggingface/hub/datasets--McAuley-Lab--Amazon-Reviews-2023/snapshots/2b6d039ed471f2ba5fd2acb718bf33b0a7e5598e/raw/review_categories/Toys_and_Games.jsonl'

In [5]:
def parse_2023(path):
    with open(path, 'r') as fp:
        for line in tqdm(fp):
            l = json.loads(line.strip())
            yield l
    
def load_2023(path):
    data = []
    with open(path, 'r') as fp:
        for line in tqdm(fp):
            data.append(json.loads(line.strip()))
    return data

In [25]:
def filter_amazon_dataset(category, metadata=False):
    rprint("Reading Ratings...")
    raw_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", 
                       f"raw_meta_{category}" if metadata else f"raw_review_{category}", 
                       trust_remote_code=True)
    raw_df = raw_dataset['full'].to_pandas()
    # df_stats(raw_df, f"{category} Reviews DataFrame Stats")
    
    rprint("Reading Ratings...")
    rating_file = hf_hub_download(
            repo_id='McAuley-Lab/Amazon-Reviews-2023',
            filename=f'benchmark/5core/rating_only/{category}.csv',
            repo_type='dataset'
            )
    rating_df = pd.read_csv(rating_file)
    # df_stats(rating_df, f"{category} Rating DataFrame Stats")
    
    # create sets for filtering
    valid_users = set(rating_df['user_id'].unique())
    valid_items = set(rating_df['parent_asin'].unique())
    
    rprint("Filtering Data...")
    # filter reviews where both user_id and parent_asin are in the 5-core subset
    if metadata:
        filtered_data = raw_df[raw_df['parent_asin'].isin(valid_items)]
    else:
        filtered_data = raw_df[
            (raw_df['user_id'].isin(valid_users)) & 
            (raw_df['parent_asin'].isin(valid_items))
        ]
    
    # df_stats(filtered_reviews, f"{category} Filtered Reviews DataFrame Stats")
    rprint("Filtered Dataset Shape:", filtered_data.shape)
    
    return filtered_data

In [28]:
category = "All_Beauty"
# category = "Sports_and_Outdoors"
# category = "Toys_and_Games"

filtered_data = filter_amazon_dataset(category, metadata=False)
df_stats(filtered_data)

Reading Ratings...

Reading Ratings...

Filtering Data...

Filtered Dataset Shape:
(2581, 10)

DataFrame shape: (2581, 10)

                                 DataFrame Stats                                 
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Column            ┃ Non-Null Count ┃ Unique Count ┃ Null/NA Count ┃ Data Type ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ rating            │ 2581           │ 5            │ 0             │ float64   │
│ title             │ 2581           │ 2363         │ 0             │ object    │
│ text              │ 2581           │ 2500         │ 0             │ object    │
│ images            │ 2581           │ Error        │ 0             │ object    │
│ asin              │ 2581           │ 372          │ 0             │ object    │
│ parent_asin       │ 2581           │ 356          │ 0             │ object    │
│ user_id           │ 2581           │ 253          │ 0             │ object    │
│ timestamp         │ 2581           │ 2507         │ 0             │ int64     │
│ helpful_vote      │ 2581           │ 21           │ 0             │ int64     │
│ verified_purchase │ 2581           │ 2            │ 0             │ bool      │
└───────────────────┴────────────────┴──────────────┴───────────────┴───────────┘

In [ ]:
DATASET_DIR = "/home/scur2745/RecSys/dataset/amazon/2023/toys"
# filtered_reviews.to_json(f'{DATASET_DIR}/reviews.json.gz', 
#                          orient='records', 
#                          lines=True,
#                          compression='gzip')

In [50]:
import pandas as pd

file_path = "/home/scur2745/RecSys/dataset/amazon/2023/raw/toys/meta.json.gz"
meta_df = pd.read_json(file_path, lines=True, compression="gzip")

In [62]:
str(meta_df["categories"][0])

"['Toys & Games', 'Remote & App Controlled Vehicles & Parts', 'Remote & App Controlled Vehicles', 'Trucks']"

### Pre-Process

In [3]:
import gzip
import json
import numpy as np
import os
import os.path as osp
import pandas as pd
import polars as pl
import torch
import sys
from collections import defaultdict
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from data.preprocessing import PreprocessingMixin
from torch_geometric.data import download_google_url
from torch_geometric.data import extract_zip
from torch_geometric.data import HeteroData
from torch_geometric.data import InMemoryDataset
from torch_geometric.io import fs
from typing import Callable
from typing import List
from typing import Optional, Dict, Union
import logging

# fetch logger
logger = logging.getLogger("recsys_logger")

In [36]:
# global vars
DATASET_DIR = "../dataset/amazon/2023/raw"
DATASET_SPLIT = "sports"

In [37]:
item_data = pd.read_csv(f"{DATASET_DIR}/{DATASET_SPLIT}/item_data.csv")
df_stats(item_data)

/scratch-local/67558/ipykernel_3408752/3368663462.py:1: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  item_data = pd.read_csv(f"{DATASET_DIR}/{DATASET_SPLIT}/item_data.csv")


DataFrame shape: (156235, 18)

                                DataFrame Stats                                
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Column          ┃ Non-Null Count ┃ Unique Count ┃ Null/NA Count ┃ Data Type ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ main_category   │ 141394         │ 35           │ 14841         │ object    │
│ title           │ 156226         │ 149035       │ 9             │ object    │
│ average_rating  │ 156235         │ 33           │ 0             │ float64   │
│ rating_number   │ 156235         │ 6577         │ 0             │ int64     │
│ features        │ 156235         │ 133479       │ 0             │ object    │
│ description     │ 156235         │ 80649        │ 0             │ object    │
│ price           │ 94131          │ 15119        │ 62104         │ object    │
│ images          │ 156235         │ 152802       │ 0             │ object    │
│ videos          │ 156235         │ 72358        │ 0             │ object    │
│ store           │ 154830         │ 34840        │ 1405          │ object    │
│ categories      │ 156235         │ 2150         │ 0             │ object    │
│ details         │ 156235         │ 154668       │ 0             │ object    │
│ asin            │ 156235         │ 156235       │ 0             │ object    │
│ bought_together │ 0              │ 0            │ 156235        │ float64   │
│ subtitle        │ 16             │ 14           │ 156219        │ object    │
│ author          │ 6              │ 6            │ 156229        │ object    │
│ brand           │ 156226         │ 27342        │ 9             │ object    │
│ id              │ 156235         │ 156235       │ 0             │ int64     │
└─────────────────┴────────────────┴──────────────┴───────────────┴───────────┘

In [38]:
df = item_data.copy()

df["img_hi_res"] = df["images"].apply(lambda x: eval(x).get("hi_res"))
df["img_large"] = df["images"].apply(lambda x: eval(x).get("large"))

df["img_hi_res"] = df["img_hi_res"].apply(lambda lst: next((x for x in lst if x is not None), None))
df["img_large"] = df["img_large"].apply(lambda lst: next((x for x in lst if x is not None), None))

df["imUrl"] = df["img_hi_res"]
df["imUrl"] = df["imUrl"].fillna(df["img_large"])

df["ext"] = df["imUrl"].str.split(".").str[-1].tolist()
df["ext"].value_counts()

ext
jpg    156096
gif       127
png         2
Name: count, dtype: int64

In [ ]:
import glob

category = "software"


files = glob.glob(f"/home/scur2745/RecSys/dataset/amazon/2023/raw/{category}/product_images/*.jpg")
len(files)

In [12]:
from collections import defaultdict

url_to_indices = defaultdict(list)

for idx, url in df["imUrl"].items():
    url_to_indices[url].append(idx)

# Filter and print only the URLs that have duplicates
for url, indices in url_to_indices.items():
    if len(indices) > 1:
        print(f"{url}: {indices}")

https://m.media-amazon.com/images/I/B1zbKPw36UL.png: [5, 3724]
https://m.media-amazon.com/images/I/71ler1wpyxL.png: [8, 16722]
https://m.media-amazon.com/images/I/A1dbmEdwmsL.png: [51, 8600]
https://m.media-amazon.com/images/I/714Qv+IRiDL.png: [113, 8284, 12684]
https://m.media-amazon.com/images/I/A1zsr1B9SzL.png: [137, 12440]
https://m.media-amazon.com/images/I/713iHmxqxZL._AC_SL1500_.jpg: [143, 8108]
https://m.media-amazon.com/images/I/A1u0ZhoCyIL.png: [169, 13643]
https://m.media-amazon.com/images/I/91uhv278I7L.png: [182, 6986]
https://m.media-amazon.com/images/I/613oMtrHS8L.jpg: [186, 10029]
https://m.media-amazon.com/images/I/71ROoE-6MhL._AC_SL1427_.jpg: [216, 1776]
https://m.media-amazon.com/images/I/41dV4TQ49FL._AC_.jpg: [238, 16345]
https://m.media-amazon.com/images/I/81eVb1BaISL._AC_SL1500_.jpg: [255, 13761]
https://m.media-amazon.com/images/I/61aW0cBM20L.jpg: [271, 14796]
https://m.media-amazon.com/images/I/71LZ34PJBzL.png: [284, 10431]
https://m.media-amazon.com/images/I/71D